# Quick implementation - motifs ordering

## Maria Inês Silva
## 28/01/2019

***

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [4]:
cwd = os.getcwd()
data_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'data'))
output_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'outputs/experiments_v3'))

In [5]:
def load_motif_data(folder):
    ts_1d = pickle.load(open(os.path.join(folder, "ts_1d.p"), "rb"))
    mdl_cost_list = pickle.load(open(os.path.join(folder, "mdl_cost.p"), "rb"))
    motif_point_list = pickle.load(open(os.path.join(folder, "motif_point.p"), "rb"))
    motif_center_list = pickle.load(open(os.path.join(folder, "motif_center.p"), "rb"))
    pattern_list = pickle.load(open(os.path.join(folder, "patterns.p"), "rb"))
    mean_dist = pickle.load(open(os.path.join(folder, "mean_dist.p"), "rb"))
    return ts_1d, mdl_cost_list, motif_point_list, motif_center_list, pattern_list, mean_dist

def load_ts_data(folder):
    trip_df = pd.read_pickle(os.path.join(folder, 'trip_df.pkl'))
    multi_dim_ts = trip_df[['ay', 'az']]
    events_ts = trip_df['event_type']
    return multi_dim_ts, events_ts

## Data import

In [6]:
multi_dim_ts, events_ts = load_ts_data(data_folder)

folder = os.path.join(output_folder, 'trip_exp1')
ts_1d, mdl_cost_list, motif_point_list, motif_center_list, pattern_list, mean_dist = load_motif_data(folder)

## Computing the events partition for each motif

In [7]:
def compute_event_partition(motif_point_list, events_ts):
    motif_event_partition_list = []
    for motif_point in motif_point_list:
        motif_events_dic_list = []
        for member in motif_point:
            member_events_list = [events_ts[i] for i in member]
            member_events_dic = {'brakes': np.sum([member_event==1 for member_event in member_events_list]),
                                 'turns': np.sum([member_event==2 for member_event in member_events_list]),
                                 'accelerations': np.sum([member_event==3 for member_event in member_events_list])}
            motif_events_dic_list.append(member_events_dic)
        motif_events_partition = pd.DataFrame(motif_events_dic_list).mean()
        motif_event_partition_list.append(motif_events_partition)
    motif_partition_df = pd.DataFrame(motif_event_partition_list)
    return motif_partition_df

In [8]:
motif_partition_df = compute_event_partition(motif_point_list, events_ts)
motif_partition_df.head()

,accelerations,brakes,turns
0,0.022222,0.000000,0.022222
1,0.048780,0.048780,0.097561
2,0.055556,0.000000,0.000000
3,0.027027,0.027027,0.054054
4,0.000000,0.000000,0.050847


## Create motif table

In [13]:
def create_motif_table(pattern_list, motif_point_list, mdl_cost_list, mean_dist, events_ts):
    motif_dic = {'pattern' : pattern_list,
                 'pattern_len' : [len(pattern) for pattern in pattern_list],
                 'n_members' : [len(temp_motif) for temp_motif in motif_point_list],
                 'mdl_cost' : mdl_cost_list,
                 'mean_dist' : mean_dist}
    motif_partition_df = compute_event_partition(motif_point_list, events_ts)
    motif_df = pd.DataFrame(motif_dic) \
                        .join(motif_partition_df) \
                        .assign(mdl_cost = lambda x: x['mdl_cost'].round(1)) \
                        .sort_values('mdl_cost')
    return motif_df

In [14]:
motif_df = create_motif_table(pattern_list, motif_point_list, mdl_cost_list, mean_dist, events_ts)
motif_df.head()

,pattern,pattern_len,n_members,mdl_cost,mean_dist,accelerations,brakes,turns
138,[acdd],1,128,60371.1,0.094457,0.023438,0.039062,0.039062
31,[bbce],1,113,61278.9,0.093627,0.008850,0.035398,0.044248
16,[edba],1,108,61480.2,0.119420,0.046296,0.018519,0.037037
135,[ddca],1,105,61811.7,0.116655,0.000000,0.028571,0.047619
34,[abde],1,100,62243.3,0.112964,0.010000,0.040000,0.030000
